In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="llama3-70b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002261C4B1A50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002261C4B2EC0>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)

In [8]:
vectorstore

In [11]:
vectorstore.similarity_search("cat")

[Document(id='cf5e4a42-34ef-48e1-9989-0fbc332e5184', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='dc620ce3-7664-406c-b240-8f2fbb427d72', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='dc3242f3-c0d5-43ca-a69e-8cebab43274d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='a4bdf80a-cff3-4dcf-b2c0-3759714bce29', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [12]:
# Async query

await vectorstore.asimilarity_search("cat")

[Document(id='cf5e4a42-34ef-48e1-9989-0fbc332e5184', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='dc620ce3-7664-406c-b240-8f2fbb427d72', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='dc3242f3-c0d5-43ca-a69e-8cebab43274d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='a4bdf80a-cff3-4dcf-b2c0-3759714bce29', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [13]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='cf5e4a42-34ef-48e1-9989-0fbc332e5184', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='dc620ce3-7664-406c-b240-8f2fbb427d72', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='dc3242f3-c0d5-43ca-a69e-8cebab43274d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='a4bdf80a-cff3-4dcf-b2c0-3759714bce29', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

In [14]:
# retriever
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat', 'dog'])


[[Document(id='cf5e4a42-34ef-48e1-9989-0fbc332e5184', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='dc620ce3-7664-406c-b240-8f2fbb427d72', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [15]:
# as_retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(['cat', 'dog'])

[[Document(id='cf5e4a42-34ef-48e1-9989-0fbc332e5184', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='dc620ce3-7664-406c-b240-8f2fbb427d72', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

### Put everything together

In [18]:
# RAG Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only. {question}
Context: {context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("Tell me about the dogs.")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness. That's all the information available about dogs in this context.
